In [99]:
import numpy as np
import sys

from keras import backend
from keras.engine import base_preprocessing_layer
from keras.utils import control_flow_util
from tensorflow.python.util.tf_export import keras_export
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    TimeDistributed,
    Dense,
    Activation,
    BatchNormalization,
    Input,
    Conv2D,
    MaxPool2D,
    Bidirectional,
    GRU,
    Flatten,
    Lambda,
    concatenate,
    Dropout,
)

from tensorflow.keras.optimizers import (
    Adam,
)

In [14]:
data = np.load('/tf/data/4stem_arr.npz', allow_pickle=True)
x_train = data['arr_0']
x_train = np.array(list(map(lambda x: np.array(list(x.values())), x_train)))
y_train = data['arr_1']

print(x_train.shape)
print(x_train)

(300, 5, 128, 431)
[[[[-8.00000000e+01 -5.48860207e+01 -4.85444870e+01 ... -3.17886047e+01
    -3.41888313e+01 -3.58236809e+01]
   [-8.00000000e+01 -5.73153381e+01 -5.51449127e+01 ... -4.90106277e+01
    -4.67924652e+01 -4.87259979e+01]
   [-8.00000000e+01 -5.19489403e+01 -5.16573257e+01 ... -4.42274513e+01
    -4.35986023e+01 -5.09270630e+01]
   ...
   [-8.00000000e+01 -8.00000000e+01 -8.00000000e+01 ... -8.00000000e+01
    -8.00000000e+01 -8.00000000e+01]
   [-8.00000000e+01 -8.00000000e+01 -8.00000000e+01 ... -8.00000000e+01
    -8.00000000e+01 -8.00000000e+01]
   [-8.00000000e+01 -8.00000000e+01 -8.00000000e+01 ... -8.00000000e+01
    -8.00000000e+01 -8.00000000e+01]]

  [[-4.17245407e+01 -7.57158813e+01 -6.49365158e+01 ... -8.00000000e+01
    -8.00000000e+01 -5.36412277e+01]
   [-4.09914284e+01 -7.93600998e+01 -6.96508179e+01 ... -8.00000000e+01
    -8.00000000e+01 -5.30732803e+01]
   [-4.13590469e+01 -7.20974655e+01 -6.76822052e+01 ... -6.47619476e+01
    -6.12643166e+01 -5.48294

In [104]:
## Keras Layer
class RandomTimeStretch(layers.Layer):
    def __init__(self, factor_min: float = 0.9, factor_max: float = 1.1):
        super(RandomTimeStretch, self).__init__()
        base_preprocessing_layer.keras_kpl_gauge.get_cell('RandomTimeStretch').set(True)
        self._factor_min = factor_min
        self._factor_max = factor_max


    def _time_stretch(
        self,
        spectrogram: tf.Tensor,
        factor: float = 1.0,
        method: tf.image.ResizeMethod = tf.image.ResizeMethod.BILINEAR,
    ) -> tf.Tensor:
        T = tf.shape(spectrogram)[0]
        T_ts = tf.cast(tf.cast(T, tf.float32) * factor, tf.int32)[0]
        F = tf.shape(spectrogram)[1]
        ts_spec = tf.image.resize(
            spectrogram, [T_ts, F], method=method
        )
        return tf.image.resize_with_crop_or_pad(ts_spec, T, F)


    def call(self, inputs, training=True):
        if training is None:
            training = backend.learning_phase()
            
        def random_time_stretch():
            factor = (
                tf.random.uniform(shape=(1,), seed=0) * (self._factor_max - self._factor_min) + self._factor_min
            )
            return self._time_stretch(inputs, factor=factor)
        
        output = control_flow_util.smart_cond(
            training,
            random_time_stretch,
            lambda: inputs
        )
        return output 


    def compute_output_shape(self, input_shape):
        return input_shape


    def get_config(self):
        ...


class RandomPitchShift(layers.Layer):
    def __init__(self, shift_min: float = -1.0, shift_max: float = 1.0):
        super(RandomPitchShift, self).__init__()
        base_preprocessing_layer.keras_kpl_gauge.get_cell('RandomPitchShift').set(True)
        self._shift_min = shift_min
        self._shift_max = shift_max


    def _pitch_shift(
        self,
        spectrogram: tf.Tensor,
        semitone_shift: float = 0.0,
        method: tf.image.ResizeMethod = tf.image.ResizeMethod.BILINEAR,
    ) -> tf.Tensor:
        spectrogram_shape = spectrogram.get_shape().as_list()
        T = spectrogram_shape[1]
        F = spectrogram_shape[2]
        factor = 2 ** (semitone_shift / 12.0)
        F_ps = F * factor
        print(F_ps)
        return tf.image.resize(
            spectrogram, [128, 500], method=method
        )
        paddings = [[0, 0], [0, tf.maximum(0, F - F_ps)], [0, 0]]
        return tf.pad(ps_spec[:, :F, :], paddings, "CONSTANT")


    def call(self, inputs, training=True):
        if training is None:
            training = backend.learning_phase()
        
        def random_pitch_shift():
            semitone_shift = (
                tf.random.uniform(shape=(1,), seed=0) * (self._shift_max - self._shift_min) + self._shift_min
            )
            
            batch = []
            for spectograms in inputs:
                collection = []
                for spectogram in spectograms:
                    print(spectogram.get_shape().as_list())
                    augmented = self._pitch_shift(spectogram[tf.newaxis, ..., tf.newaxis], semitone_shift=semitone_shift)
                    print(augmented.get_shape().as_list())
                    collection.append()
                batch.append(collection)
            return batch
  
        output = control_flow_util.smart_cond(
            training,
            random_pitch_shift,
            lambda: inputs
        )
        return output 


    def compute_output_shape(self, input_shape):
        return input_shape


    def get_config(self):
        ...


In [105]:
Input_Layer = tf.keras.layers.Input((5,128,431))
Data_Augmentation = RandomPitchShift()(Input_Layer)


# RNN Block
#BiRNN_Block = TimeDistributed(Bidirectional(GRU(128)))(Data_Augmentation)
#BiRNN_Block = Dropout(0.5)(BiRNN_Block)
#blub = Flatten()(BiRNN_Block)

# CNN Block
#CNN_Block = tf.keras.layers.Reshape((128,431,5))(Data_Augmentation)
#CNN_Block = Conv2D(
#    filters=16,
#    kernel_size=[3, 3],
#    padding='same',
#)(CNN_Block)
#CNN_Block = Activation('relu')(CNN_Block)
#CNN_Block = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(CNN_Block)
#CNN_Block = Flatten()(CNN_Block)

#Final_Classification_Block = concatenate([
#    blub,
#    CNN_Block,
#], axis=-1)
#Output_Layer = Dense(16, activation='softmax')(Final_Classification_Block)

model = Model(Input_Layer, Data_Augmentation)

opt = Adam(learning_rate=0.002)
model.compile(
    loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy']
)

model.summary()

[128, 431]
Tensor("random_pitch_shift_37/while/while/mul:0", shape=(1,), dtype=float32)
[1, 128, 500, 1]
Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_47 (InputLayer)        [(None, 5, 128, 431)]     0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [77]:
image = tf.constant([
 [1,0,0,0,0],
 [0,1,0,0,0],
 [0,0,1,0,0],
 [0,0,0,1,0],
 [0,0,0,0,1],
])

image = image[tf.newaxis, ..., tf.newaxis]

tf.image.resize(image, [3,5])[0,...,0].numpy()

array([[0.6666667, 0.3333333, 0.       , 0.       , 0.       ],
       [0.       , 0.       , 1.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.3333335, 0.6666665]],
      dtype=float32)

In [62]:
model.fit(
    x_train,
    y_train,
    epochs=50,
    batch_size=10,
    verbose=1
)

Epoch 1/50
Tensor("model_12/random_pitch_shift_15/while/resize/Squeeze:0", shape=(128, None, 5), dtype=float32)
[<tf.Tensor 'model_12/random_pitch_shift_15/while/Pad:0' shape=(128, None, 5) dtype=float32>]


InaccessibleTensorError: in user code:

    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:838 run_step  **
        outp

(128, 431, 5)
(128, 431, 5)
